# Lanchain OpenAI Setup

In [1]:
import langchain
import os
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Import chat templates
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Caching
from langchain.cache import InMemoryCache
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

In [2]:
cacheType = 'in_memory'

if cacheType == 'in_memory':
    set_llm_cache(InMemoryCache())
elif cacheType == 'sqlite':
    set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# Set OpenAI API key and create LLM and Chat LLM. Note that key can be stored in a separate file or as an environment variable. Refer to docs.
api_key = open('./openai_key.txt').read()
os.environ['OPENAI_API_KEY'] = api_key
llm = OpenAI(openai_api_key=api_key)
chat = ChatOpenAI(openai_api_key=api_key, temperature=0)

In [3]:
# Import wikipedia loader
from langchain.document_loaders import WikipediaLoader

In [4]:
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

C:\ProgramData\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\ProgramData\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [5]:
len(documents)

24

In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)
print(len(docs))
print(docs[0])


Created a chunk of size 504, which is longer than the specified 500
Created a chunk of size 528, which is longer than the specified 500


52
page_content="Project MKUltra was an illegal human experimentation program designed and undertaken by the U.S. Central Intelligence Agency (CIA) and intended to develop procedures and identify drugs that could be used during interrogations to weaken people and force confessions through brainwashing and psychological torture. It began in 1953 and was halted in 1973. MKUltra used numerous methods to manipulate its subjects' mental states and brain functions, such as the covert administration of high doses of psychoactive drugs (especially LSD) and other chemicals without the subjects' consent, electroshocks, hypnosis, sensory deprivation, isolation, verbal and sexual abuse, and other forms of torture.MKUltra was preceded by Project Artichoke. It was organized through the CIA's Office of Scientific Intelligence and coordinated with the United States Army Biological Warfare Laboratories. The program engaged in illegal activities, including the use of U.S. and Canadian citizens as unwitt

In [8]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [9]:
embedding_function = OpenAIEmbeddings()

In [10]:
db = Chroma.from_documents(docs, embedding_function, persist_directory='./some_new_mkultra_db')
db.persist()

# Multi Query

In [11]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=chat)

In [12]:
# Enable logging
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [13]:
question = "When was this declassiied?"


response = retriever_from_llm.get_relevant_documents(question)
len(response)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of the declassification?', '2. Can you provide the specific time when this was declassified?', '3. Do you have information on the declassification date?']


6

In [14]:
response[0].page_content

'The Church Committee (formally the United States Senate Select Committee to Study Governmental Operations with Respect to Intelligence Activities) was a US Senate select committee in 1975 that  investigated abuses by the Central Intelligence Agency (CIA), National Security Agency (NSA), Federal Bureau of Investigation (FBI), and the Internal Revenue Service (IRS). Chaired by Idaho Senator Frank Church (D-ID), the committee was part of a series of investigations into intelligence abuses in 1975, dubbed the "Year of Intelligence", including its House counterpart, the Pike Committee, and the presidential Rockefeller Commission. The committee\'s efforts led to the establishment of the permanent US Senate Select Committee on Intelligence.\nThe most shocking revelations of the committee include Operation MKULTRA, which involved the drugging and torture of unwitting US citizens as part of human experimentation on mind control; COINTELPRO, which involved the surveillance and infiltration of A